**Model 1 – Baseline Linear Pricing**

In [ ]:
# ---------------------------------- MODEL 1: BASELINE LINEAR PRICING ----------------------------------

# Install required packages (only once)
!pip install pathway bokeh panel --quiet

# ----------------------------- Imports & Preprocessing -----------------------------
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn

# Load dataset and clean
df = pd.read_csv("dataset.csv")
df.columns = df.columns.str.strip()
df = df.rename(columns={"SystemCodeNumber": "LotID"})

# Combine timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

# Prepare data for Model 1
df = df[["Timestamp", "LotID", "Occupancy", "Capacity"]]
df.to_csv("parking_stream.csv", index=False)

# ----------------------------- Pathway Streaming Schema -----------------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    LotID: str
    Occupancy: int
    Capacity: int

# Simulate real-time stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    LotID = data.LotID
)

# ----------------------------- Model 1 Logic -----------------------------
BASE_PRICE = 10.0
ALPHA = 2.0

@pw.udf
def baseline_price(occupancy: int, capacity: int) -> float:
    if capacity == 0:
        return BASE_PRICE
    price = BASE_PRICE + ALPHA * (occupancy / capacity)
    return max(5, min(20, round(price, 2)))

price_stream = data_with_time.with_columns(
    price = baseline_price(
        data_with_time.Occupancy,
        data_with_time.Capacity
    )
)

pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Model 1: Baseline Linear Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.scatter("t", "price", source=source, size=6, color="blue")  # fixed
    return fig

viz = price_stream.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
import sys
import contextlib
with contextlib.redirect_stdout(sys.stdout):
    pw.run()

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input      finished                                     
    PythonRead…     finished          7838        18298    output         6162       None                          
    PythonRead…     finished          7786        18220                                                            
    PythonRead…     finished          7857        18308     Above you can see the latency of input and output      
    PythonRead…     finished          7813        18291    operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    